In [1]:
import sys
sys.path.append('..')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import cupy as _cu
from tqdm.auto import tqdm
import numpy as np
import math

In [4]:
import scaredcu

/home/toluntosun/miniconda3/envs/sca/lib/python3.12/site-packages/cupy/_creation/from_data.py:53: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of 3562152210 to int32 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  return _core.array(obj, dtype, copy, order, subok, ndmin, blocking)
/home/toluntosun/miniconda3/envs/sca/lib/python3.12/site-packages/cupy/_creation/from_data.py:53: DeprecationWarning: NumPy will stop allowing conversion of out-of-bound Python integers to integer arrays.  The conversion of -3562152210 to int32 will fail in the future.
For the old behavior, usually:
    np.array(value).astype(dtype)
will give the desired result (the cast overflows).
  return _core.array(obj, dtype, copy, order, subok, ndmin, blocking)
/home/toluntosun/miniconda3/envs/sca/lib/python3.12/site-packages/cupy/_creation/from_dat

In [5]:
import estraces

In [6]:
ths = estraces.read_ths_from_ets_file(f'kyber_basemult_montgomery_pon_t0.ets')

In [7]:
import scaredcu.gpu_format as gpu_format

ths_ = gpu_format.read_ths_from_ram(_cu.array(ths.samples)[:,:], c=_cu.array(ths.c)[:,:],
                                      s=_cu.array(ths.s)[:,:])

In [8]:
## from kyber_reverse.ipynb
REDUCTION = 'MONTGOMERY'
if REDUCTION == 'MONTGOMERY':

    S0L, S0R, S1L, S1R = 93, 9053, 9189, 18149
    C0L, C0R, C1L, C1R = 76, 80, 128, 132
    
    K = 140
    X = 4

elif REDUCTION == 'PLANTARD':

    S0L, S0R, S1L, S1R = 128, 9600, 9740, 19212
    C0L, C0R, C1L, C1R = 53, 57, 101, 105
    
    K = 148
    X = 0


def make_frame(key_index, d, is_high=True):
    x = 0 if is_high else X    
    T = S0L if d == 0 else S1L
    if key_index % 2 == 0:
        return range(T + K*(key_index//2) + C0L + x, T + K*(key_index//2) + C0R + x)
    else:
        return range(T + K*(key_index//2) + C1L + x, T + K*(key_index//2) + C1R + x)


In [9]:
def print_scores(ki, scores, guesses, scores_sorted=None, l=10):
    if scores_sorted is None:
        scores_sorted = scores.argsort()[::-1]

    for i in range(l):
        print(ki, i, guesses[scores_sorted[i]], scores[scores_sorted[i]])
    print()

In [10]:
step = 900
batch_size = 50
key_index = 0
scaredcu.container.set_batch_size(batch_size)

model = scaredcu.models.AbsoluteValue()
selection_function = scaredcu.lbc.kyber.selection_functions.KyberBaseMulIterated(cu_step=1<<19, words=[key_index],
                                                                                 low=False, high=True, neg_trick=True)

In [11]:
selection_function.reset()

for _ in tqdm(range(selection_function.num_steps)):

    assert selection_function.done() == False


    cpa = scaredcu.CPAAttackAlternative(selection_function=selection_function,
                                            model=model,
                                            discriminant=scaredcu.maxabs,
                                            convergence_step=None)


    mean = ths_.samples.mean(axis=0)
    pp_ = scaredcu.preprocesses.high_order.CenteredProduct(mean=mean, frame_1=make_frame(key_index, 0, True),
                                                            frame_2=make_frame(key_index, 1, True), mode='same')

    for i in range(0, 900, step):
        container = scaredcu.Container(ths_[i:i + step], preprocesses=[pp_])
        cpa.run(container)
    selection_function.save_scores(cpa.scores)
    print(cpa.scores.shape, cpa.scores.argmax(), selection_function.guesses[cpa.scores.argmax()], selection_function.scores.shape)
    print_scores(key_index, selection_function.scores[:,0], selection_function.np_guesses)
    selection_function.next()

  0%|          | 0/11 [00:00<?, ?it/s]

(524288, 1) 0 [0 0] (524288, 1)
0 0 [0 0] nan
0 1 [2452   91] 0.19873405
0 2 [1056   98] 0.17012143
0 3 [2699  134] 0.15882051
0 4 [783 125] 0.15727833
0 5 [2244   83] 0.15257357
0 6 [1439  133] 0.15247917
0 7 [1874  128] 0.15066567
0 8 [1151   28] 0.15061736
0 9 [420  82] 0.15039805

(524288, 1) 180014 [1883  211] (1048576, 1)
0 0 [0 0] nan
0 1 [1883  211] 0.2862216
0 2 [2452   91] 0.19873405
0 3 [1608  248] 0.17811412
0 4 [1056   98] 0.17012143
0 5 [119 289] 0.16774932
0 6 [506 209] 0.16201416
0 7 [2115  259] 0.15963006
0 8 [2831  311] 0.15947346
0 9 [2699  134] 0.15882051

(524288, 1) 351459 [1855  420] (1572864, 1)
0 0 [0 0] nan
0 1 [1883  211] 0.2862216
0 2 [2452   91] 0.19873405
0 3 [1855  420] 0.18084382
0 4 [1608  248] 0.17811412
0 5 [1056   98] 0.17012143
0 6 [119 289] 0.16774932
0 7 [964 428] 0.16728127
0 8 [506 209] 0.16201416
0 9 [2115  259] 0.15963006

(524288, 1) 23401 [1674  479] (2097152, 1)
0 0 [0 0] nan
0 1 [1883  211] 0.2862216
0 2 [2452   91] 0.19873405
0 3 [1855  4